## Challenge 1: Combining & Cleaning Data

In [2]:
import pandas as pd

# URLs
file1_url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv"
file2_url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv"
file3_url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv"

# Read CSVs
df1 = pd.read_csv(file1_url)
df2 = pd.read_csv(file2_url)
df3 = pd.read_csv(file3_url)

# Optional: quick preview
print(df1.head(), df2.head(), df3.head())


  Customer          ST GENDER             Education Customer Lifetime Value  \
0  RB50392  Washington    NaN                Master                     NaN   
1  QZ44356     Arizona      F              Bachelor              697953.59%   
2  AI49188      Nevada      F              Bachelor             1288743.17%   
3  WW63253  California      M              Bachelor              764586.18%   
4  GA49547  Washington      M  High School or Below              536307.65%   

    Income  Monthly Premium Auto Number of Open Complaints     Policy Type  \
0      0.0                1000.0                    1/0/00   Personal Auto   
1      0.0                  94.0                    1/0/00   Personal Auto   
2  48767.0                 108.0                    1/0/00   Personal Auto   
3      0.0                 106.0                    1/0/00  Corporate Auto   
4  36357.0                  68.0                    1/0/00   Personal Auto   

   Vehicle Class  Total Claim Amount  
0  Four-Door Car 

## Step 1: Clean the datasets

In [3]:
def clean_df(df):
    # Strip column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    # Remove duplicates
    df = df.drop_duplicates()
    # Fill missing values (example: fill numeric NaN with 0, string NaN with 'Unknown')
    for col in df.select_dtypes(include='number').columns:
        df[col] = df[col].fillna(0)
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].fillna('Unknown')
    return df


In [4]:
#Apply cleaning
df1_clean = clean_df(df1)
df2_clean = clean_df(df2)
df3_clean = clean_df(df3)


C:\Users\kyila\AppData\Local\Temp\ipykernel_59716\3053997771.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(0)
C:\Users\kyila\AppData\Local\Temp\ipykernel_59716\3053997771.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna('Unknown')


In [5]:
#Combine datasets
combined_df = pd.concat([df1_clean, df2_clean, df3_clean], ignore_index=True)


## Challenge 2- structuring data

In [6]:
marketing_url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
marketing_df = pd.read_csv(marketing_url)

# Quick clean: lowercase column names and strip spaces
marketing_df.columns = marketing_df.columns.str.strip().str.lower().str.replace(' ', '_')


In [7]:
#Total revenue per sales channel
# Pivot table: total revenue per sales channel
revenue_by_channel = pd.pivot_table(
    marketing_df,
    values='total_claim_amount',  # replace with revenue column if different
    index='sales_channel',        # column indicating branch, web, call center, mail
    aggfunc='sum'
).round(2)

print(revenue_by_channel)


               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04


In [8]:
# Average customer lifetime value per gender and education
avg_clv = pd.pivot_table(
    marketing_df,
    values='customer_lifetime_value',  # numeric column
    index='gender',
    columns='education',
    aggfunc='mean'
).round(2)

print(avg_clv)


education  Bachelor  College   Doctor  High School or Below   Master
gender                                                              
F           7874.27  7748.82  7328.51               8675.22  8157.05
M           7703.60  8052.46  7415.33               8149.69  8168.83


In [10]:
#Number of complaints by policy type and month
# Count complaints by policy_type and month
complaints_summary = marketing_df.groupby(['policy_type', 'month']).size().reset_index(name='num_complaints')

# This is already in long format:
print(complaints_summary)


      policy_type  month  num_complaints
0  Corporate Auto      1            1252
1  Corporate Auto      2            1089
2   Personal Auto      1            4329
3   Personal Auto      2            3799
4    Special Auto      1             237
5    Special Auto      2             204
